# Modelling Electric Vehicle Consumption

## Import Libaries

In [151]:
import pandas as pd
import os
import numpy as np
from numpy import inf
import datetime
from datetime import datetime, timedelta
import descartes
import geopy.distance
from pathlib import Path

## Load Data

In [152]:
LoadData = pd.read_csv('UnitMain_9.csv')
LoadData.head()

,Date,Time,Latitude,Longitude,Altitude,Speed,Heading,Signal Quality,Number of Satalites Connected,X-Axis Acceleration,Y-Axis Acceleration,Z-Axis Acceleration
0,12/4/2022,7:20:20,-33.89134,18.95951,235.9,1.09,37.18,2,9,0.194,-0.022,1.008
1,12/4/2022,7:20:22,-33.89134,18.95952,235.9,1.09,36.87,2,9,0.154,-0.032,1.010
2,12/4/2022,7:20:24,-33.89134,18.95952,235.9,0.41,36.87,2,9,0.134,0.064,1.004
3,12/4/2022,7:20:25,-33.89134,18.95952,235.9,0.33,36.87,2,9,0.068,0.096,1.024
4,12/4/2022,7:20:27,-33.89134,18.95952,235.9,0.31,36.87,2,9,0.072,0.114,1.020


## Calculations

In [153]:
def readFile():
    journey = pd.read_csv('UnitMain_9.csv') #should come right from the database
    # Join date and time columns into one column
    journey['DateTime'] = pd.to_datetime(journey['Date'] + journey['Time'], format = '%m/%d/%Y%H:%M:%S')

    # Error dection 1- Check if the time logger got stuck for a second, and correct if so
    for i in range(len(journey) - 1):
        if journey['DateTime'][i] == journey['DateTime'][i + 1]:
            journey['DateTime'][i + 1] += timedelta(seconds = 1)

    # Convert speed in km/h to m/s 
    #neglect small velocities for instances like rolling etc
    # Reccomadations- consider velocities that are too high
    journey['Velocity'] = np.where(journey['Speed'] >= 0.5, journey['Speed']/3.6, 0)

    #Calculate elevation change
    #Reccomadation:  Wont need this as device gives angle
    journey['ElevChange'] = np.where(abs(journey['Altitude'].shift(-1) - journey['Altitude']) >= 0.2, journey['Altitude'].shift(-1) - journey['Altitude'], 0)

    # Calculate time between samples. Useful for kinetic model. 
    # This is needed because some samples are 1/2Hz. 
    journey['DeltaT'] = journey['DateTime'].shift(-1) - journey['DateTime']
    journey.DeltaT = journey['DeltaT']/ np.timedelta64(1, 's') # Convert from timedelta to float

    # Calculate change in velocity between each timestep 
    journey['DeltaV'] = journey['Velocity'].shift(-1) - journey['Velocity']

    # Calculate acceleration
    journey['Acceleration'] = np.where(journey['DeltaT'] > 0, journey['DeltaV']/journey['DeltaT'], 0)

    # Joins lat/lon Coords into one column, useful for getDist function
    journey['Coordinates'] = list(zip(journey.Latitude, journey.Longitude))
    return journey


## Raw data

In [154]:
'''reads the data files using the readFile function, 
and returns a nested dictionary containing all trip files '''

def rawData():
    rawTrips = {} # Dictionary to store trip in as dataframes
    thisTrip = readFile()
    unit= 'Device A' #consider multple devices
    rawTrips[unit]={}
    rawTrips[unit] = thisTrip

    return rawTrips

## Slope and distance functions

In [155]:
# Inputs: dataframe with vehicle journey data
# Will need distance not slope
def getDistSlope(journey):
    #initilise
    Distance = np.zeros(len(journey))
    Slope = np.zeros(len(journey))
    l_route = journey.shape[0]

    for i in range(0,l_route-1): 
        elev_change = journey['Altitude'][i + 1] - journey['Altitude'][i]
        if abs(elev_change) < 0.2:
            elev_change = 0
        
        dist_lateral = geopy.distance.geodesic(journey.Coordinates.iloc[i],  # (m) Lateral distance between two lat/lon coord pairs
                                       journey.Coordinates.iloc[i+1]).m
        dist_3d = np.sqrt(dist_lateral**2 + elev_change**2)  # (m) 3D Distance using pythogoras to account for elevation change
        if i == 0:
            Distance[i] = 0
        else:
            Distance[i] = dist_3d
        if Distance[i] != 0 and elev_change != 0:
            Slope[i] = np.arcsin(elev_change/dist_3d)
    journey['Displacement_m'] = list(Distance) # Used for computing kWh/km 
    journey['slope_rad'] = list(Slope) 

## External forces functions

In [156]:
# Rolling Resistance (road friction) (N)
def getRollingResistance(mass, c_rr, slope, vel, grav = 9.81): #c_rr is coeff of rolling resistance
    rr = 0
    if vel > 0.3:
        rr = -mass * grav * c_rr * np.cos(slope)
    return rr
 
# Aerodynamic Drag Force (N)
def getAerodynamicDrag(c_d, A, vel, rho = 1.184): # rho is air density 20C, c_d is air drag coeff
    return -0.50 * rho * c_d * A * vel**2

# Road Slope Force (N)
def getRoadSlopeDrag(mass, slope, grav = 9.81):
    return -mass * grav * np.sin(slope)

    ## Wheel rotation, engine rotation etc
    

In [157]:
class Drivecycle: 
    """
    Inputs: dataframe with journey info
    """
    def __init__(self, journey): 
        self.displacement = journey.Displacement_m # m
        self.velocity = journey.Velocity # m/s
        self.acceleration = journey.Acceleration #m/s^2
        self.slope = journey.slope_rad # rad
        self.dt = journey.DeltaT # Time elapsed between each timestamp
        self.dv = journey.DeltaV
        

## Vehicle class

In [158]:
class Vehicle:
    """
    Inputs: Physical parameters of vehicle for modeling energy consumption
    Returns an array of vehicle energy consumption at each timestamp """

    def __init__(self, mass, cd, crr, A, eff, rgbeff, p0):

        # Vehicle physical parameters
        self.mass = mass # (kg) Total veh weight including payload
        self.crr = crr # Rolling resistance coeff
        self.cd = cd  # Air drag coeff
        self.A = A # (m^2) Approximation of vehicle frontal area 
        self.eff = eff # Powertrain efficiency
        self.rgbeff = rgbeff # Recuperative braking efficiency
        self.p0 = p0 # (W) Constant power loss (to power auxiliary vehicle systems)

    # Computes energy expenditure from a Drivecycle object
    def getEnergyExpenditure(self,cycle):
        
        v = cycle.velocity # (m/s) 
        s = cycle.slope # (rad)
        a = cycle.acceleration # (m/s^2)
        dt = cycle.dt # (s)
        d = cycle.displacement # (m)
        dv = cycle.dv # (m/s)
            
            
        Er = [] # Total energy consumption (J), from battery if positive, to battery if negative (RGbrake)
                
        for slope,vel,acc,delta_t, delta_v in zip(s,v,a,dt,dv):
            
            force, frr, fad, fsd = 0,0,0,0
            
            if vel != 0: #No drag if velocity = 0
                frr = getRollingResistance(self.mass,self.crr, slope, vel) # (N)
                fad = getAerodynamicDrag(self.cd, self.A, vel) # (N)
                fsd = getRoadSlopeDrag(self.mass, slope) # (N)
                
            force = frr + fad + fsd # (N)
            
            exp_speed_delta = force * delta_t / self.mass # (N-s/kg)
            
            unexp_speed_delta = delta_v - exp_speed_delta # (m/s)
            
            try:
                prop_brake_force = self.mass * unexp_speed_delta / delta_t # (kg * m/s)/(s) = (N)
            
                kinetic_power = prop_brake_force * vel # (N)*(m/s) = (W)
                
                propultion_work = kinetic_power * delta_t  # (W)*(s) = (J) 
                
            except ZeroDivisionError:
                prop_brake_force = 0
                kinetic_power = 0
                propultion_work = 0

            Er.append(propultion_work) # (J)

        ErP = [0.0]*len(Er)
        ErB = [0.0]*len(Er)

        for i in range(len(Er)):            
            if Er[i] > 0: # Propulsion Energy 
                ErP[i] = Er[i]/self.eff # (J)
                
            elif Er[i] < 0: # Recuperated Energy (from regen braking system)
                ErB[i]= Er[i]*self.rgbeff # (J)
    
    
    
        return ErP,ErB 

## Process data

In [159]:
def processData(trip_dict):
    processed_trips = trip_dict #initialize dictionary of same structure
    for unit,journey in trip_dict.items(): #loop through values(dataframe)
                
                print("Processing journey:", unit)
                p0=100
                minibus = Vehicle(mass = 3900, cd = 0.36, crr = 0.02, A = 4.0, eff = 0.90, rgbeff = 0.65, p0 = 100)
                
                
                
                getDistSlope(journey)
                drvcycle = Drivecycle(journey)

                #drvcycle= getDistSlope(journey) # Augment journey dataframe with distance and slope columns, Create drivecycle object with trip data
                
                # Compute energy expenditure (with and without regenerative braking)
                journey['Propultion Work (J)'],journey['Braking Work (J)'] = minibus.getEnergyExpenditure(drvcycle)
                journey['Offload Work (J)'] = journey['DeltaT'] * p0 #  (J)

                journey['Energy Consumption (kWh)'] = (journey['Propultion Work (J)'] + journey['Braking Work (J)'] + journey['Offload Work (J)'])/3.6e6 # (kWh)                
            
                #Drop duplicate and intermediate calculation columns to reduce clutter
                journey = journey.drop(['DeltaT', 'slope_rad', 'Velocity', 'DeltaV',
                                       'Acceleration', 'Coordinates', 'DateTime', 'ElevChange'], axis = 1)
                
                #Insert processed journey into dictionary
                processed_trips[unit] = journey
                
    return processed_trips

## Calling read and process functions

In [160]:
rawTrips = rawData()
processed_trips = processData(rawTrips)

Processing journey: Device A


## Analyse

In [161]:
#summary statistics on energy consumptions 
energy_consumption = []

for unit,journey_df in processed_trips.items():
    energy_consumption.append(journey_df['Energy Consumption (kWh)'].sum()/(journey_df['Displacement_m'].sum()/1e3))

summ_df = pd.DataFrame({'Energy Consumption (kWh)': pd.Series(energy_consumption)})

print(round(summ_df.describe(),2))

#Write to csv file- or send to database
summ_df.to_csv('EnergyConsumption.csv', index=False)

       Energy Consumption (kWh)
count                      1.00
mean                       0.32
std                         NaN
min                        0.32
25%                        0.32
50%                        0.32
75%                        0.32
max                        0.32
